# Multinomial Naive Bayes

## Importación de librerías

In [167]:
import pandas as pd
import numpy as np
import re as re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk
from nltk.corpus import stopwords

## Lectura de los datos

Leemos los datos que usaremos para entrenar nuestro algorítmo

### Lectura

In [168]:
data = pd.read_csv('train.csv',usecols = ['text','target'])

### Primer vistazo

In [169]:
data

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


## Limpieza de los datos

Realizamos una limpieza del texto que se corresponde a los tweets para obtener un mejor resultado a la hora de procesar los datos

### Definicion de funciones auxiliares

In [170]:
#Eliminar numeros de un texto
def eliminar_numeros(text):
    return re.sub("\d+", "",text)

#Eliminar puntuacion
def eliminar_puntuacion(text):
    return re.sub(r'[^\w\s]','',text)

#Pasar letras a minusculas
def minusculas(text):
    return text.lower()

#Eliminar caracteres especiales
def eliminar_caracteres(text):
    return re.sub('[^a-zA-Z0-9 \n\.]', '',text)

#Eliminar urls
def eliminar_url(text):
    url_reg = re.compile(r'https?://\S+|www\.\S+')
    return url_reg.sub(r'',text)

#Eliminar palabras comunes como articulos,pronombres,preposiciones
def eliminar_comunes(text):
    comunes = stopwords.words('english')
    newtext = ''
    for word in text.split():
        if word not in comunes:
            newtext += ' ' + word
    return newtext

### Aplicamos las funciones a nuestros datos

In [171]:
data['text'] = data['text'].apply(eliminar_numeros)
data['text'] = data['text'].apply(eliminar_puntuacion)
data['text'] = data['text'].apply(minusculas)
data['text'] = data['text'].apply(eliminar_caracteres)
data['text'] = data['text'].apply(eliminar_url)
data['text'] = data['text'].apply(eliminar_comunes)

### Salida

In [172]:
data

,text,target
0,deeds reason earthquake may allah forgive us,1
1,forest fire near la ronge sask canada,1
2,residents asked shelter place notified office...,1
3,people receive wildfires evacuation orders ca...,1
4,got sent photo ruby alaska smoke wildfires po...,1
...,...,...
7608,two giant cranes holding bridge collapse near...,1
7609,ariaahrary thetawniest control wild fires cal...,1
7610,utckm volcano hawaii httptcozdtoydebj,1
7611,police investigating ebike collided car littl...,1


## Representando el texto de los tweets como una matriz de datos numéricos

Llamaremos token a las palabras que conforman el texto de los tweets. Podemos entonces crear una matriz cuyas columnas sean los tokens que aparecen en todos los tweets y las filas cada uno de los tweets. Sean aij los elementos de la matriz. El elemento aij representa la cantidad de veces que aparece el token j en el tweet i.

### Ejemplo

Tomemos tres tweets cualesquiera.

In [173]:
ejemplo = ['hola mi nombre mi federico', 'me gusta mi nombre', 'mi nombre nombre es federico']

In [174]:
vect = CountVectorizer(analyzer='word',binary=False)

In [175]:
vect.fit(ejemplo)

CountVectorizer()

In [176]:
vect.get_feature_names()

['es', 'federico', 'gusta', 'hola', 'me', 'mi', 'nombre']

In [177]:
ejemplo_dtm = vect.transform(ejemplo)
ejemplo_dtm

<3x7 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [178]:
ejemplo_array = ejemplo_dtm.toarray()

In [179]:
pd.DataFrame(ejemplo_array,columns = vect.get_feature_names())

,es,federico,gusta,hola,me,mi,nombre
0,0,1,0,1,0,2,1
1,0,0,1,0,1,1,1
2,1,1,0,0,0,1,2


### Observación

Como la mayoría de los tweets utilizan un conjunto reducido de tokens con respecto al total de tokens con el que trabajaremos, se espera que la mayoría de los elementos de la matriz sean cero.

## Armando la matriz para nuestros datos

### Particionando los datos

Particionamos nuestros datos en dos ya que una parte de ellos será utilizada para entrenar a nuestro algorítmo y la otra será utilizada para probarlo. En nuestro caso las variables X respresentan a el texto de los tweets e Y el target asociado a los tweets. La intención de nuestro algorítmo es dado un determinado texto de un tweet(x) poder predecir su target(y)

In [180]:
X = data['text']
Y = data['text']

In [181]:
X_train,X_test,y_train,y_test= train_test_split(data.text, data.target, test_size=0.013,random_state=1)

In [182]:
len(X_train)

7514

In [183]:
len(X_test)

99

### Instanciando el vectorizador

In [184]:
vect = CountVectorizer(analyzer='word',binary=False)

### Obtenemos una lista con todos los tokens obtenidos de los tweets

In [185]:
vect.fit(X_train)

CountVectorizer()

In [186]:
vect.get_feature_names()

['aa',
 'aaaa',
 'aaaaaaallll',
 'aaaaaand',
 'aaarrrgghhh',
 'aaceorg',
 'aal',
 'aampb',
 'aampw',
 'aan',
 'aannnnd',
 'aar',
 'aaronthefm',
 'aashiqui',
 'ab',
 'aba',
 'abandon',
 'abandoned',
 'abandonedpics',
 'abandoning',
 'abbandoned',
 'abbott',
 'abbruchsimulator',
 'abbswinston',
 'abbyairshow',
 'abc',
 'abcchicago',
 'abceyewitness',
 'abcnews',
 'abcnorio',
 'abcs',
 'abe',
 'aberdeen',
 'aberdeenfanpage',
 'aberdeenfc',
 'aberystwythshrewsbury',
 'abes',
 'abha',
 'abia',
 'ability',
 'abject',
 'ablaze',
 'able',
 'ableg',
 'abninfvet',
 'aboard',
 'abomb',
 'abombed',
 'abomination',
 'abortion',
 'abortions',
 'abouts',
 'abrancaballero',
 'abs',
 'absence',
 'absolute',
 'absolutely',
 'absolutsumya',
 'abstorm',
 'abstract',
 'absurd',
 'absurdly',
 'abubaraa',
 'abuse',
 'abused',
 'abuseddesolateamplost',
 'abuses',
 'abusing',
 'abysmaljoiner',
 'ac',
 'acaciapenn',
 'academia',
 'acarewornheart',
 'acc',
 'accept',
 'accepte',
 'accepts',
 'access',
 'accident

### Armamos la matriz

In [187]:
X_train_dtm = vect.transform(X_train)
X_train_dtm

<7514x21153 sparse matrix of type '<class 'numpy.int64'>'
	with 71063 stored elements in Compressed Sparse Row format>

In [188]:
X_train_array = X_train_dtm.toarray()

In [189]:
pd.DataFrame(X_train_array,columns = vect.get_feature_names())

,aa,aaaa,aaaaaaallll,aaaaaand,aaarrrgghhh,aaceorg,aal,aampb,aampw,aan,...,zoom,zotar,zouma,zourryart,zrnf,zss,zumiez,zurich,zxathetis,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7509,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7510,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7511,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7512,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


*** OBS: Notemos que la matriz tiene tantas filas como cantidad de tweets y tantas columnas como tokens. En este caso se encontraron 17413 tokens. Podemos notar como la mayría de los elementos de la matriz son cero como mencionamos anteriormente ***

## Entrenando el modelo

Usaremos el clasificador multinomial Naive Bayes para realizar  la clasificación

In [190]:
nb = MultinomialNB()

Entrenamos el modelo

In [191]:
%time nb.fit(X_train_dtm, y_train)

CPU times: user 2.62 ms, sys: 3.06 ms, total: 5.68 ms
Wall time: 4.99 ms


MultinomialNB()

## Probamos el modelo

### Sobre el set de train

In [192]:
predicciones_train = nb.predict(X_train_dtm)

In [193]:
metrics.accuracy_score(y_train,predicciones_train)

0.9230769230769231

### Sobre el set de test

In [194]:
X_test_dtm = vect.transform(X_test)
X_test_dtm

<99x21153 sparse matrix of type '<class 'numpy.int64'>'
	with 709 stored elements in Compressed Sparse Row format>

In [195]:
predicciones_test = nb.predict(X_test_dtm)

In [196]:
metrics.accuracy_score(y_test,predicciones_test)

0.8686868686868687

## Preparando el submit de Kaggle

### Leemos los datos

In [197]:
submit= pd.read_csv('test.csv',usecols = ['text'])

In [198]:
submit

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,Storm in RI worse than last hurricane. My city...
3260,Green Line derailment in Chicago http://t.co/U...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...


### Limpiamos los datos

In [199]:
submit['text'] = submit['text'].apply(eliminar_numeros)
submit['text'] = submit['text'].apply(eliminar_puntuacion)
submit['text'] = submit['text'].apply(minusculas)
submit['text'] = submit['text'].apply(eliminar_url)
submit['text'] = submit['text'].apply(eliminar_comunes)

In [200]:
submit

,text
0,happened terrible car crash
1,heard earthquake different cities stay safe e...
2,forest fire spot pond geese fleeing across st...
3,apocalypse lighting spokane wildfires
4,typhoon soudelor kills china taiwan
...,...
3258,earthquake safety los angeles ûò safety faste...
3259,storm ri worse last hurricane cityampothers h...
3260,green line derailment chicago httptcoutbxlcbiuy
3261,meg issues hazardous weather outlook hwo http...


### Predecimos

In [201]:
texts = vect.transform(submit['text'])
predicciones_kaggle = nb.predict(texts)
submit['target'] = predicciones_kaggle
#submit.to_csv('SUBMITS/submission-onehot.csv',index=False)